In [10]:
import sympy as sp
from IPython.display import display

sp.init_printing()
I3 = sp.eye(3)

# --- Rotations utilitaires ---
def Rx(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
def Ry(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
def Rz(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
def euler_R(roll,pitch,yaw,order="ZYX"):
    if order.upper()=="ZYX":  return Rz(yaw)*Ry(pitch)*Rx(roll)
    if order.upper()=="XYZ":  return Rx(roll)*Ry(pitch)*Rz(yaw)
    raise ValueError("ordre = 'ZYX' ou 'XYZ'")

def rotation_from_two_vectors(a, b):
    """R ∈ SO(3) tel que R a = b, pour a,b unitaires (gère cas a=±b)."""
    a = sp.Matrix(a); b = sp.Matrix(b)
    if (a-b).norm() < 1e-12: return I3
    if (a+b).norm() < 1e-12:
        # 180° autour d'un axe ⟂ à a
        if abs(a[0]) < 0.9: axis = sp.Matrix([1,0,0]).cross(a)
        else:               axis = sp.Matrix([0,1,0]).cross(a)
        axis = axis/axis.norm()
        vx,vy,vz = axis
        Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
        return I3 + 2*Vx*Vx
    v = a.cross(b); s = v.norm(); c = (a.T*b)[0]
    v = v/s
    vx,vy,vz = v
    Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
    # Rodrigues avec angle = arccos(c)
    ang = sp.acos(c)
    return I3 + sp.sin(ang)*Vx + (1-sp.cos(ang))*(Vx*Vx)

def any_perp(u):
    """Unitaire n ⟂ u."""
    u = sp.Matrix(u); 
    trial = sp.Matrix([1,0,0]) if abs(u[0])<0.9 else sp.Matrix([0,1,0])
    n = u.cross(trial); n = n/n.norm()
    return n

# --- Conception purement rotationnelle d'un pas pour une translation cible t_world* = Δt v d_hat ---
def design_step_SO3(e, E_ext, i, j, t_world_star):
    """
    Entrées:
      - e (vielbein 3x3, p.ex. diag anisotrope)
      - E_ext ∈ SO(3) (bloc "translation externe")
      - i,j ∈ {1,2,3} (sonde f = (e_i, e_j))
      - t_world_star (3x1) = Δt v d_hat (déplacement monde désiré AU PAS)
    Sorties:
      - SYX, SYY ∈ SO(3) tels que Y_eff = SYX e_i + SYY e_j = r u_local,
        avec r = ||t_world_star|| (borné par 2) et u_local = E_ext^T e^{-1} d_hat / ||...||
    """
    e_inv = e.inv()
    r = t_world_star.norm()
    if r == 0:
        return I3, I3, sp.zeros(3,1)  # pas de translation
    d_hat = t_world_star / r
    u_local = E_ext.T * (e_inv * d_hat)
    u_local = u_local / u_local.norm()

    # Choisir deux unitaires a,b tels que a+b = r u_local, avec r ∈ [0,2]
    if r > 2:
        raise ValueError("Pas rotationnel pur: ||t_world_star|| doit être ≤ 2 (fixe Δt=2/c).")
    theta = sp.acos(r/2)             # r = 2 cos(theta)
    n = any_perp(u_local)
    a =  sp.cos(theta)*u_local + sp.sin(theta)*n
    b =  sp.cos(theta)*u_local - sp.sin(theta)*n

    # Construit SYX, SYY en SO(3) tels que SYX e_i = a, SYY e_j = b
    e1,e2,e3 = sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])
    basis = [e1,e2,e3]
    ei, ej = basis[i-1], basis[j-1]
    SYX = rotation_from_two_vectors(ei, a)
    SYY = rotation_from_two_vectors(ej, b)

    # Vérification: Y_eff = a + b, t_world = e E_ext Y_eff = r d_hat = t_world_star
    Y_eff = a + b
    t_world = e * (E_ext * Y_eff)
    return SYX, SYY, t_world

# --- Un pas de dynamique (X,Y au sens "mesuré" via sonde f=(e_i,e_j)) ---
def one_step(e, R_ext, R_int, E_ext, E_int, SXX, SXY, SYX, SYY, i, j):
    """
    Applique V1 = M (S f), f=(e_i,e_j). Retourne X1_world, Y1_loc, et les objets utiles.
    """
    # Opérateur physique et état-opérateur
    M  = sp.Matrix.vstack(sp.Matrix.hstack(R_ext, E_ext),
                          sp.Matrix.hstack(E_int, R_int))
    S  = sp.Matrix.vstack(sp.Matrix.hstack(SXX, SXY),
                          sp.Matrix.hstack(SYX, SYY))
    # Sonde
    e1,e2,e3 = sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])
    f = sp.Matrix.vstack([e1,e2,e3][i-1], [e1,e2,e3][j-1])

    # Etat effectif initial, action, et lecture monde
    V0 = S * f
    V1 = M * V0
    X1_loc = V1[:3,:]
    Y1_loc = V1[3:,:]
    X1_world = e * X1_loc
    return X1_world, Y1_loc, V0, V1, S, M

# ================== DÉMO PARAMÉTRABLE ==================
# Constantes
c = sp.Symbol('c', positive=True)   # vitesse limite
Delta_t = 2/c                       # calibration: v_max = 2/Δt = c

# Vielbein (non-euclidien global, mais hors des blocs)
e = sp.diag(2, 1, sp.Rational(1,2))

# Blocs "physique" (tous dans SO(3); constants ici)
R_ext = euler_R(sp.pi/7, sp.pi/6, sp.pi/5,  "ZYX")
R_int = euler_R(sp.pi/10, sp.pi/9, -sp.pi/8,"ZYX")
E_ext = euler_R(sp.pi/13,-sp.pi/12, sp.pi/11,"ZYX")
E_int = euler_R(-sp.pi/16, sp.pi/15, sp.pi/14,"ZYX")

# Blocs "champ" S — tu peux les fixer (SO(3)); SYX,SYY seront re-designés à chaque pas par design_step_SO3
SXX = euler_R(sp.pi/9,  -sp.pi/10, sp.pi/18,"ZYX")
SXY = euler_R(-sp.pi/12, sp.pi/11,  sp.pi/8,"ZYX")

# Sonde: couple (e_i, e_j)
i, j = 1, 3   # (e1, e3)

# Séquence de vitesses/directions cibles (exemple)
# On donne des directions monde (unitaires) et des vitesses v_k ≤ c; ici on prend v=c, c/2, c
d_list = [sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])]
v_list = [c, c/2, c]

# Exécution multi-pas
X_world = sp.Matrix([0,0,0])
print("Δt choisi =", Delta_t)
for k,(d_hat,v) in enumerate(zip(d_list, v_list), start=1):
    t_star = Delta_t * v * d_hat              # déplacement monde AU PAS (borne v ≤ c ⇒ ||t_star|| ≤ 2)
    SYX_k, SYY_k, t_world_k = design_step_SO3(e, E_ext, i, j, t_star)
    X1_world, Y1_loc, V0, V1, S, M = one_step(e, R_ext, R_int, E_ext, E_int,
                                              SXX, SXY, SYX_k, SYY_k, i, j)
    X_world += t_world_k                      # comme R_ext agit sur X_eff, mais la translation monde vaut bien t_star
    print(f"\n--- Pas {k} ---")
    print("t_star (monde) :"); display(sp.simplify(t_star))
    print("t_world obtenu :"); display(sp.simplify(t_world_k))
    print("Vitesse par pas v = ||t||/Δt  :"); display(sp.simplify(t_world_k.norm()/Delta_t))

print("\nPosition monde accumulée (somme des translations) :"); display(sp.simplify(X_world))


Δt choisi = 2/c

--- Pas 1 ---
t_star (monde) :


⎡2⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

t_world obtenu :


⎡4⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

Vitesse par pas v = ||t||/Δt  :


...

t_world obtenu :


⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

Vitesse par pas v = ||t||/Δt  :



Position monde accumulée (somme des translations) :


⎡4⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣1⎦

In [9]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# -------------------- Rotations utilitaires --------------------
def Rx(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
def Ry(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
def Rz(a): c,s = sp.cos(a), sp.sin(a); return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
def euler_R(roll,pitch,yaw,order="ZYX"):
    if order.upper()=="ZYX":  return Rz(yaw)*Ry(pitch)*Rx(roll)
    if order.upper()=="XYZ":  return Rx(roll)*Ry(pitch)*Rz(yaw)
    raise ValueError("ordre = 'ZYX' ou 'XYZ'")

def ok_SO3(R, tol=1e-10):
    RtR = (R.T*R).evalf(); det = sp.N(R.det())
    return (RtR - sp.eye(3)).norm() < tol and abs(det - 1) < tol

# -------------------- Construction opérateurs --------------------
def build_M(Rext,Rint,Eext,Eint):
    return sp.Matrix.vstack(sp.Matrix.hstack(Rext,Eext),
                            sp.Matrix.hstack(Eint,Rint))
def build_S(SXX,SXY,SYX,SYY):
    return sp.Matrix.vstack(sp.Matrix.hstack(SXX,SXY),
                            sp.Matrix.hstack(SYX,SYY))

# -------------------- Rotation qui envoie a -> b (unitaires non colinéaires) --------------------
def rotation_from_two_vectors(a, b):
    """Construit une rotation R (SO3) telle que R*a = b (a et b unitaires)."""
    a = sp.Matrix(a); b = sp.Matrix(b)
    # Cas dégénérés (a==b ou a==-b)
    if (a-b).norm() < 1e-14: 
        return sp.eye(3)
    if (a+b).norm() < 1e-14:
        # Rotation de pi autour d'un axe orthogonal à a
        # On choisit un axe simple :
        if abs(a[0]) < 0.9: axis = sp.Matrix([1,0,0]).cross(a)
        else:               axis = sp.Matrix([0,1,0]).cross(a)
        axis = axis/axis.norm()
        vx,vy,vz = axis
        Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
        return sp.eye(3) + 2*Vx*Vx  # rotation 180°
    # Rodrigues
    v = a.cross(b); s = v.norm(); c = (a.T*b)[0]
    v = v/s
    vx,vy,vz = v
    Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
    return sp.eye(3) + sp.sin(sp.acos(c))*Vx + (1-c)*(Vx*Vx)

# -------------------- Design de la colonne j de SYY pour une cible t_world*, avec sonde (e_i,e_j) --------------------
def design_SYY_column_for_probe_pair(e, Eext, SYX, i, j, t_world_star, strict_SO3=True):
    """
    Sonde f=(e_i,e_j). On veut t_world = e*(Eext * Y_eff) = t_world_star,
    avec Y_eff = SYX*e_i + SYY*e_j  =>  SYY*e_j = Yreq := Eext^T e^{-1} t_world* - SYX*e_i.
    - strict_SO3=True : impose SYY ∈ SO(3).
        -> on aligne la colonne j de SYY avec y_unit = Yreq/||Yreq||, via une rotation R telle que R*e_j = y_unit.
        -> amplitude non libre (||Y_eff|| dépend de la somme des colonnes).
    - strict_SO3=False : SYY = rho * R, avec rho = ||Yreq||, R*e_j = y_unit.
    Retourne SYY (rotation pure ou rho*rotation) et rho.
    """
    e_inv = e.inv()
    e1, e2, e3 = sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])
    basis = [e1,e2,e3]
    ei, ej = basis[i-1], basis[j-1]
    Yreq = Eext.T * (e_inv * t_world_star) - (SYX * ei)
    norm = Yreq.norm()
    if norm == 0:
        # Pas de besoin dans la colonne j (liberté totale) : on renvoie I
        return sp.eye(3), sp.Integer(1)
    y_unit = Yreq / norm
    Rmap = rotation_from_two_vectors(ej, y_unit)   # Rmap * ej = y_unit
    if strict_SO3:
        return Rmap, sp.Integer(1)
    else:
        return norm * Rmap, norm

# -------------------- Scénario avec sonde (e_i,e_j) --------------------
def run_scenario_with_probe_pair(e, Rext,Rint,Eext,Eint, 
                                 SXX,SXY,SYX,SYY, i, j, n_digits=8):
    e_inv = e.inv()
    e1, e2, e3 = sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])
    basis = [e1,e2,e3]
    # Sonde f = (e_i, e_j)
    x_ref, y_ref = basis[i-1], basis[j-1]
    f = sp.Matrix.vstack(x_ref, y_ref)

    # Vérifs SO(3)
    print("=== Vérifs SO(3) ===")
    for name,R in [("R_ext",Rext),("R_int",Rint),("E_ext",Eext),("E_int",Eint),
                   ("SXX",SXX),("SXY",SXY),("SYX",SYX)]:
        print(f"{name} ∈ SO(3) ? -> {ok_SO3(R)}")
    print("SYY rotation pure ? ->", ok_SO3(SYY))

    # Opérateurs
    M = build_M(Rext,Rint,Eext,Eint)
    S = build_S(SXX,SXY,SYX,SYY)

    # Etat effectif initial (particule := champ sondé)
    V0 = S * f
    Xeff_loc, Yeff_loc = V0[:3,:], V0[3:,:]

    # Physique locale
    V1 = M * V0
    X1_loc, Y1_loc = V1[:3,:], V1[3:,:]

    # Lecture monde
    R_world = e * Rext * e_inv
    Xeff_world = e * Xeff_loc
    t_world = e * (Eext * Yeff_loc)
    X1_world = e * X1_loc

    # Décomposition monde
    X1_world_pred = R_world * Xeff_world + t_world
    ok = bool(X1_world_pred.evalf(n_digits).equals(X1_world.evalf(n_digits)))

    # ---- Affichages utiles ----
    print("\n=== Vielbein e et e^{-1} ==="); display(e); display(e_inv)
    print("\n=== M (6x6) ==="); display(M)
    print("\n=== S (6x6) ==="); display(S)
    print(f"\n=== Sonde f = (e_{i}, e_{j}) ==="); display(f)

    print("\n=== Etat effectif V0 = S f ===")
    print("X_eff (local):"); display(Xeff_loc)
    print("Y_eff (local):"); display(Yeff_loc)
    print("X_eff (world):"); display(Xeff_world)

    print("\n=== Application V1 = M V0 ===")
    print("X1_loc:"); display(X1_loc)
    print("Y1_loc:"); display(Y1_loc)

    print("\n=== Décomposition monde ===")
    print("R_world = e R_ext e^{-1}:"); display(R_world)
    print("t_world = e (E_ext Y_eff):"); display(t_world)
    print("R_world*X_eff_world + t_world:"); display(X1_world_pred)
    print("Correspond à X1_world ? ->", ok)

    return dict(ok=ok, X1_world=X1_world, t_world=t_world, R_world=R_world,
                Xeff_world=Xeff_world, Yeff_loc=Yeff_loc)

# ===================== Paramètres d’exemple : tout en SO(3) =====================

# Monde non-euclidien (anisotrope)
e = sp.diag(2, 1, sp.Rational(1,2))

# 4 rotations indépendantes pour M
R_ext = euler_R(sp.pi/7,  sp.pi/6,  sp.pi/5,   "ZYX")
R_int = euler_R(sp.pi/10, sp.pi/9, -sp.pi/8,   "ZYX")
E_ext = euler_R(sp.pi/13,-sp.pi/12, sp.pi/11,  "ZYX")   # "translation" externe = rotation
E_int = euler_R(-sp.pi/16, sp.pi/15, sp.pi/14, "ZYX")   # "translation" interne = rotation

# 4 rotations indépendantes pour S
SXX = euler_R(sp.pi/9,  -sp.pi/10, sp.pi/18,  "ZYX")
SXY = euler_R(-sp.pi/12, sp.pi/11,  sp.pi/8,  "ZYX")
SYX = euler_R(sp.pi/15,  sp.pi/7,  -sp.pi/13, "ZYX")
# SYY sera conçu ci-dessous selon la cible t_world*, en fonction du couple (i,j)

# ----- Choix de la sonde (e_i, e_j) -----
i, j = 1, 3   # ex : (e1, e3)

# ----- Cible de translation monde -----
t_world_star = sp.Matrix([1, 2, 3])

# ----- Conception de SYY (colonne j) pour atteindre t_world*, mode strict SO(3) OU avec gain -----
SYY_strict, rho_strict = design_SYY_column_for_probe_pair(e, E_ext, SYX, i, j, t_world_star, strict_SO3=True)
print("Mode strict SO(3) (amplitude non libre) — rho implicite = 1")
res_strict = run_scenario_with_probe_pair(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY_strict, i, j, n_digits=8)

SYY_gain, rho_gain = design_SYY_column_for_probe_pair(e, E_ext, SYX, i, j, t_world_star, strict_SO3=False)
print("\nMode avec gain (amplitude libre) — rho =", rho_gain)
res_gain = run_scenario_with_probe_pair(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY_gain, i, j, n_digits=8)


Mode strict SO(3) (amplitude non libre) — rho implicite = 1
=== Vérifs SO(3) ===
R_ext ∈ SO(3) ? -> True
R_int ∈ SO(3) ? -> True
E_ext ∈ SO(3) ? -> True
E_int ∈ SO(3) ? -> True
SXX ∈ SO(3) ? -> True
SXY ∈ SO(3) ? -> True
SYX ∈ SO(3) ? -> True
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Zara\anaconda3\envs\ceremap\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Zara\AppData\Local\Temp\ipykernel_6732\3228559879.py", line 166, in <module>
    res_strict = run_scenario_with_probe_pair(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY_strict, i, j, n_digits=8)
  File "C:\Users\Zara\AppData\Local\Temp\ipykernel_6732\3228559879.py", line 92, in run_scenario_with_probe_pair
    print("SYY rotation pure ? ->", ok_SO3(SYY))
  File "C:\Users\Zara\AppData\Local\Temp\ipykernel_6732\3228559879.py", line 16, in ok_SO3
    RtR = (R.T*R).evalf(); det = sp.N(R.det())
  File "c:\Users\Zara\anaconda3\envs\ceremap\lib\site-packages\sympy\matrices\matrixbase.py", line 3080, in det
    return _det(self, method=method, iszerofunc=iszerofunc)
  File "c:\Users\Zara\anaconda3\envs\ceremap\lib\site-packages\sympy\matrices\determinant.py", line 695, in 

In [8]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# ---------- Rotations utilitaires ----------
def Rx(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
def Ry(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
def Rz(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
def euler_R(roll,pitch,yaw,order="ZYX"):
    if order.upper()=="ZYX":  return Rz(yaw)*Ry(pitch)*Rx(roll)
    if order.upper()=="XYZ":  return Rx(roll)*Ry(pitch)*Rz(yaw)
    raise ValueError("ordre = 'ZYX' ou 'XYZ'")

def ok_SO3(R, tol=1e-10):
    RtR = (R.T*R).evalf()
    det = sp.N(R.det())
    return (RtR - sp.eye(3)).norm() < tol and abs(det - 1) < tol

# ---------- Construction opérateurs ----------
def build_M(Rext,Rint,Eext,Eint):
    return sp.Matrix.vstack(
        sp.Matrix.hstack(Rext,Eext),
        sp.Matrix.hstack(Eint,Rint)
    )
def build_S(SXX,SXY,SYX,SYY):
    return sp.Matrix.vstack(
        sp.Matrix.hstack(SXX,SXY),
        sp.Matrix.hstack(SYX,SYY)
    )

# ---------- Conception de SYY pour viser t_world^* ----------
def design_SYY_for_target(e, Eext, t_world_star, strict_SO3=True):
    """
    Sonde triviale f0=(0,e3). On veut t_world = e*(Eext * (SYY * e3)) = t_world_star.
    Notons  y_req_loc = Eext^T * e^{-1} * t_world_star.
    - strict_SO3=True : impose SYY ∈ SO(3). On choisit R_YY s.t. R_YY e3 = y_req_unit.
                        L'amplitude ||t_world|| ne peut pas être ajustée (à e fixé).
    - strict_SO3=False : SYY = rho * R_YY, avec rho = ||y_req_loc||.
    Retourne SYY et info (rho).
    """
    e_inv = e.inv()
    y_req_loc = (Eext.T) * (e_inv * t_world_star)  # vecteur local requis
    norm = sp.sqrt((y_req_loc.T * y_req_loc)[0])
    if norm == 0:
        # pas de translation requise => SYY peut être I
        return sp.eye(3), sp.Integer(1)

    y_unit = y_req_loc / norm

    # Construire une rotation R_YY telle que R_YY * e3 = y_unit
    e3 = sp.Matrix([0,0,1])
    v = e3.cross(y_unit)
    s = sp.sqrt((v.T*v)[0])
    c = (e3.T*y_unit)[0]
    if s == 0:
        # Déjà aligné (y_unit == e3 ou == -e3)
        if c > 0:
            R_YY = sp.eye(3)
        else:
            # rotation de pi autour de x (par ex) pour envoyer e3 -> -e3
            R_YY = Rx(sp.pi)
    else:
        vx,vy,vz = v
        Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
        R_YY = sp.eye(3) + (Vx) + (Vx*Vx)*((1-c)/(s**2))

    if strict_SO3:
        return R_YY, sp.Integer(1)       # amplitude non ajustée
    else:
        rho = sp.simplify(norm)          # ajuste l'amplitude
        return rho*R_YY, rho

# ---------- Scénario complet avec sonde triviale ----------
def run_scenario(e, Rext,Rint,Eext,Eint, SXX,SXY,SYX,SYY, n_digits=8):
    e_inv = e.inv()
    # Sonde triviale : f0 = (x_ref=0, y_ref=e3)
    x_ref = sp.Matrix([0,0,0])
    y_ref = sp.Matrix([0,0,1])
    f0 = sp.Matrix.vstack(x_ref, y_ref)

    # Vérifs SO(3)
    for name,R in [("R_ext",Rext),("R_int",Rint),("E_ext",Eext),("E_int",Eint),
                   ("SXX",SXX),("SXY",SXY),("SYX",SYX)]:
        print(f"{name} ∈ SO(3) ? -> {ok_SO3(R)}")
    # SYY peut être rho*R, donc pas forcément SO(3) si rho≠1
    print(f"SYY rotation pure ? -> {ok_SO3(SYY)}")

    # Opérateurs
    M = build_M(Rext,Rint,Eext,Eint)
    S = build_S(SXX,SXY,SYX,SYY)

    # Etat effectif initial (particule := champ sondé)
    V0 = S * f0
    Xeff_loc, Yeff_loc = V0[:3,:], V0[3:,:]

    # Physique (locale)
    V1 = M * V0
    X1_loc, Y1_loc = V1[:3,:], V1[3:,:]

    # Lecture monde
    R_world = e * Rext * e_inv
    Xeff_world = e * Xeff_loc
    t_world = e * (Eext * Yeff_loc)
    X1_world = e * X1_loc

    # Décomposition monde
    X1_world_pred = R_world * Xeff_world + t_world
    ok = bool(X1_world_pred.evalf(n_digits).equals(X1_world.evalf(n_digits)))

    # ---- Affichages utiles ----
    print("\n=== Vielbein e et e^{-1} ==="); display(e); display(e_inv)
    print("\n=== M (6x6) ==="); display(M)
    print("\n=== S (6x6) ==="); display(S)
    print("\n=== Sonde triviale f0 = [0; e3] ==="); display(f0)

    print("\n=== Etat effectif V0 = S f0 ===")
    print("X_eff (local):"); display(Xeff_loc)
    print("Y_eff (local):"); display(Yeff_loc)
    print("X_eff (world):"); display(Xeff_world)

    print("\n=== Application V1 = M V0 ===")
    print("X1_loc:"); display(X1_loc)
    print("Y1_loc:"); display(Y1_loc)

    print("\n=== Décomposition monde ===")
    print("R_world = e R_ext e^{-1}:"); display(R_world)
    print("t_world = e (E_ext Y_eff):"); display(t_world)
    print("R_world*X_eff_world + t_world:"); display(X1_world_pred)
    print("Correspond à X1_world ? ->", ok)

    return dict(ok=ok, X1_world=X1_world, t_world=t_world, R_world=R_world,
                Xeff_world=Xeff_world, Yeff_loc=Yeff_loc)

# ===================== Paramètres d’exemple =====================

# Monde non-euclidien (anisotrope)
e = sp.diag(2, 1, sp.Rational(1,2))

# 4 rotations indépendantes pour M
R_ext = euler_R(sp.pi/7,  sp.pi/6,  sp.pi/5,   "ZYX")
R_int = euler_R(sp.pi/10, sp.pi/9, -sp.pi/8,   "ZYX")
E_ext = euler_R(sp.pi/13,-sp.pi/12, sp.pi/11,  "ZYX")   # "translation" externe = rotation
E_int = euler_R(-sp.pi/16, sp.pi/15, sp.pi/14, "ZYX")   # "translation" interne = rotation

# Blocs de S en rotations (SYY pourra inclure un gain si strict_SO3=False)
SXX = euler_R(sp.pi/9,  -sp.pi/10, sp.pi/18,  "ZYX")
SXY = euler_R(-sp.pi/12, sp.pi/11,  sp.pi/8,  "ZYX")
SYX = euler_R(sp.pi/15,  sp.pi/7,  -sp.pi/13, "ZYX")

# === Choix 1 : strict SO(3) ===
SYY_strict, rho1 = design_SYY_for_target(e, E_ext, t_world_star=sp.Matrix([1,2,3]), strict_SO3=True)
print("Mode strict SO(3) — rho implicite = 1")
res_strict = run_scenario(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY_strict, n_digits=8)

# === Choix 2 : avec gain rho (amplitude libre) ===
SYY_gain, rho2 = design_SYY_for_target(e, E_ext, t_world_star=sp.Matrix([1,2,3]), strict_SO3=False)
print("\nMode avec gain — rho =", rho2)
res_gain = run_scenario(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY_gain, n_digits=8)


Mode strict SO(3) — rho implicite = 1
R_ext ∈ SO(3) ? -> True
R_int ∈ SO(3) ? -> True
E_ext ∈ SO(3) ? -> True
E_int ∈ SO(3) ? -> True
SXX ∈ SO(3) ? -> True
SXY ∈ SO(3) ? -> True
SYX ∈ SO(3) ? -> True


KeyboardInterrupt: 

In [7]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# -------------------- Rotations utilitaires --------------------
def Rx(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
def Ry(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
def Rz(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
def euler_R(roll,pitch,yaw,order="ZYX"):
    if order.upper()=="ZYX":  return Rz(yaw)*Ry(pitch)*Rx(roll)
    if order.upper()=="XYZ":  return Rx(roll)*Ry(pitch)*Rz(yaw)
    raise ValueError("ordre = 'ZYX' ou 'XYZ'")

def ok_SO3(R, tol=1e-10):
    RtR = (R.T*R).evalf()
    det = sp.N(R.det())
    I = sp.eye(3)
    return (RtR - I).norm() < tol and abs(det - 1) < tol

# -------------------- Construction M et S --------------------
def build_M(Rext,Rint,Eext,Eint):
    return sp.Matrix.vstack(
        sp.Matrix.hstack(Rext,Eext),
        sp.Matrix.hstack(Eint,Rint)
    )

def build_S(SXX,SXY,SYX,SYY):
    return sp.Matrix.vstack(
        sp.Matrix.hstack(SXX,SXY),
        sp.Matrix.hstack(SYX,SYY)
    )

# -------------------- Solveur y_ref pour t_world cible --------------------
def solve_yref_for_target_t(e, Rext, Eext, SYY, SYX, Xref_world, t_world_target):
    """
    Impose t_world_target = e*(Eext * Y_eff),
    avec Y_eff = SYX*x_ref + SYY*y_ref, x_ref = e^{-1} Xref_world.
    Comme Eext ∈ SO(3), Eext^{-1} = Eext^T.
    => y_ref = SYY^{-1} ( Eext^T e^{-1} t_world_target - SYX x_ref ).
    """
    e_inv = e.inv()
    x_ref = e_inv * Xref_world
    Yeff_req = Eext.T * (e_inv * t_world_target)
    y_ref = SYY.inv() * (Yeff_req - SYX * x_ref)
    return x_ref, y_ref

# -------------------- Scénario complet --------------------
def run_scenario(e, Rext,Rint,Eext,Eint, SXX,SXY,SYX,SYY,
                 Xref_world, y_ref, n_digits=8):
    e_inv = e.inv()
    # Vérifs SO(3)
    for name,R in [("R_ext",Rext),("R_int",Rint),("E_ext",Eext),("E_int",Eint),
                   ("SXX",SXX),("SXY",SXY),("SYX",SYX),("SYY",SYY)]:
        print(f"{name} ∈ SO(3) ? -> {ok_SO3(R)}")

    # Opérateurs
    M = build_M(Rext,Rint,Eext,Eint)
    S = build_S(SXX,SXY,SYX,SYY)

    # Sonde locale
    x_ref = e_inv * Xref_world
    f = sp.Matrix.vstack(x_ref, y_ref)

    # Etat effectif initial (particule := champ sondé)
    V0 = S * f
    Xeff_loc, Yeff_loc = V0[:3,:], V0[3:,:]

    # Physique
    V1 = M * V0
    X1_loc, Y1_loc = V1[:3,:], V1[3:,:]

    # Lecture monde
    R_world = e * Rext * e_inv
    Xeff_world = e * Xeff_loc
    t_world = e * (Eext * Yeff_loc)
    X1_world = e * X1_loc

    # Décomposition monde
    X1_world_pred = R_world * Xeff_world + t_world
    ok = bool(X1_world_pred.evalf(n_digits).equals(X1_world.evalf(n_digits)))

    # ---- Affichages utiles ----
    print("\n=== Vielbein e et e^{-1} ==="); display(e); display(e_inv)
    print("\n=== Opérateur physique M (6x6) ==="); display(M)
    print("\n=== Etat-opérateur S (6x6) ==="); display(S)
    print("\n=== Sonde f = [x_ref; y_ref] (local) ==="); display(f)
    print("\n=== Etat effectif initial V0 = S f ===")
    print("X_eff (local):"); display(Xeff_loc)
    print("Y_eff (local):"); display(Yeff_loc)
    print("X_eff (world):"); display(Xeff_world)
    print("\n=== Application V1 = M V0 ===")
    print("X1_loc:"); display(X1_loc)
    print("Y1_loc:"); display(Y1_loc)
    print("\n=== Décomposition monde ===")
    print("R_world = e R_ext e^{-1}:"); display(R_world)
    print("t_world = e (E_ext Y_eff):"); display(t_world)
    print("R_world*X_eff_world + t_world:"); display(X1_world_pred)
    print("Correspond à X1_world ? ->", ok)

    return dict(ok=ok, X1_world=X1_world, t_world=t_world, R_world=R_world,
                Xeff_world=Xeff_world, Yeff_loc=Yeff_loc)

# ===================== Paramètres non triviaux (tout en SO(3)) =====================

# Monde non-euclidien
e = sp.diag(2, 1, sp.Rational(1,2))

# 4 rotations indépendantes pour M
R_ext = euler_R(sp.pi/7,  sp.pi/6,  sp.pi/5,   "ZYX")
R_int = euler_R(sp.pi/10, sp.pi/9, -sp.pi/8,   "ZYX")
E_ext = euler_R(sp.pi/13,-sp.pi/12, sp.pi/11,  "ZYX")   # "translation" externe = rotation
E_int = euler_R(-sp.pi/16, sp.pi/15, sp.pi/14, "ZYX")   # "translation" interne = rotation

# 4 rotations indépendantes pour S (SXY, SYX ∈ SO(3))
SXX = euler_R(sp.pi/9,  -sp.pi/10, sp.pi/18,  "ZYX")    # tu peux garder I3 si tu préfères
SXY = euler_R(-sp.pi/12, sp.pi/11,  sp.pi/8,  "ZYX")    # maintenant rotation
SYX = euler_R(sp.pi/15,  sp.pi/7,  -sp.pi/13, "ZYX")    # maintenant rotation
SYY = R_int                                                # rotation (SO(3)) et inversible

# Sonde : position ref monde et translation monde cible
Xref_world = sp.Matrix([3, -1, 2])
t_world_target = sp.Matrix([1, 2, 3])

# Calcule y_ref pour imposer t_world_target (avec toutes ces rotations)
x_ref, y_ref = solve_yref_for_target_t(e, R_ext, E_ext, SYY, SYX, Xref_world, t_world_target)

# Lance le scénario
res = run_scenario(e, R_ext,R_int,E_ext,E_int, SXX,SXY,SYX,SYY,
                   Xref_world, y_ref, n_digits=10)


R_ext ∈ SO(3) ? -> True
R_int ∈ SO(3) ? -> True
E_ext ∈ SO(3) ? -> True
E_int ∈ SO(3) ? -> True
SXX ∈ SO(3) ? -> True
SXY ∈ SO(3) ? -> True
SYX ∈ SO(3) ? -> True
SYY ∈ SO(3) ? -> True

=== Vielbein e et e^{-1} ===


⎡2  0   0 ⎤
⎢         ⎥
⎢0  1   0 ⎥
⎢         ⎥
⎣0  0  1/2⎦

⎡1/2  0  0⎤
⎢         ⎥
⎢ 0   1  0⎥
⎢         ⎥
⎣ 0   0  2⎦


=== Opérateur physique M (6x6) ===


⎡               ⎛1   √5⎞                                                       ↪
⎢            √3⋅⎜─ + ──⎟                                 ________              ↪
⎢               ⎝4   4 ⎠                                ╱ 5   √5     ⎛π⎞   ⎛1  ↪
⎢            ───────────                           -   ╱  ─ - ── ⋅cos⎜─⎟ + ⎜─  ↪
⎢                 2                                  ╲╱   8   8      ⎝7⎠   ⎝8  ↪
⎢                                                                              ↪
⎢                 ________                              ________               ↪
⎢                ╱ 5   √5                              ╱ 5   √5     ⎛π⎞        ↪
⎢          √3⋅  ╱  ─ - ──                             ╱  ─ - ── ⋅sin⎜─⎟        ↪
⎢             ╲╱   8   8                            ╲╱   8   8      ⎝7⎠   ⎛1   ↪
⎢          ───────────────                          ─────────────────── + ⎜─ + ↪
⎢                 2                                          2            ⎝4   ↪
⎢                           

...

t_world = e (E_ext Y_eff):


⎡                                                    ⎛                         ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎜  ⎛                  ___ ↪
⎢                                                    ⎜  ⎜                 ╱ √5 ↪
⎢                                                    ⎜  ⎜           √3⋅  ╱  ── ↪
⎢  ⎛     ⎛π ⎞    ⎛π ⎞   ⎛  √6   √2⎞    ⎛π ⎞    ⎛π ⎞⎞ ⎜  ⎜  1   √5      ╲╱   8  ↪
⎢2⋅⎜- sin⎜──⎟⋅cos⎜──⎟ + ⎜- ── + ──⎟⋅sin⎜──⎟⋅cos⎜──⎟⎟⋅⎜- ⎜- ─ + ── + ────────── ↪
⎢  ⎝     ⎝11⎠    ⎝13⎠   ⎝  4    4 ⎠    ⎝13⎠    ⎝11⎠⎠ ⎜  ⎝  8   8           2   ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎝                         ↪
⎢                                                                              ↪
⎢                                                     ⎛                        ↪
⎢                           

R_world*X_eff_world + t_world:


⎡            ⎛               ⎛             ⎛                                   ↪
⎢            ⎜               ⎜             ⎜     ⎛π ⎞    ⎛π ⎞                  ↪
⎢            ⎜               ⎜             ⎜  sin⎜──⎟⋅cos⎜──⎟                  ↪
⎢            ⎜               ⎜    ________ ⎜     ⎝11⎠    ⎝13⎠     ⎛  √6   √2⎞  ↪
⎢            ⎜      ________ ⎜  ╲╱ 2 - √2 ⋅⎜- ─────────────── + 2⋅⎜- ── + ──⎟⋅ ↪
⎢   ⎛1   √5⎞ ⎜     ╱ √2   1  ⎜             ⎝         2            ⎝  4    4 ⎠  ↪
⎢√3⋅⎜─ + ──⎟⋅⎜2⋅  ╱  ── + ─ ⋅⎜- ────────────────────────────────────────────── ↪
⎢   ⎝4   4 ⎠ ⎜  ╲╱   4    2  ⎜                                                 ↪
⎢            ⎜               ⎜                                                 ↪
⎢            ⎝               ⎝                                                 ↪
⎢───────────────────────────────────────────────────────────────────────────── ↪
⎢                                                                              ↪
⎢                           

Correspond à X1_world ? -> True


In [6]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# ---------- rotations ----------
def Rx(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
def Ry(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
def Rz(a):
    c,s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
def euler_R(roll,pitch,yaw,order="ZYX"):
    if order.upper()=="ZYX":  return Rz(yaw)*Ry(pitch)*Rx(roll)
    if order.upper()=="XYZ":  return Rx(roll)*Ry(pitch)*Rz(yaw)
    raise ValueError("ordre 'ZYX' ou 'XYZ'")

def is_SO3(R, tol=sp.nsimplify(1e-10)):
    return ( (R.T*R).equals(sp.eye(3)) and sp.simplify(R.det()-1)==0 )

# ---------- construction opérateurs ----------
def build_S(SXX,SXY,SYX,SYY):
    return sp.Matrix.vstack(sp.Matrix.hstack(SXX,SXY),
                            sp.Matrix.hstack(SYX,SYY))

def build_M(Rext,Rint,Eext,Eint):
    return sp.Matrix.vstack(sp.Matrix.hstack(Rext,Eext),
                            sp.Matrix.hstack(Eint,Rint))

# ---------- solveur y_ref pour t_world voulu ----------
def yref_for_target_translation(e, Rext, Eext, S_YY, S_YX, Xref_world, t_world_target):
    e_inv = e.inv()
    xref = e_inv * Xref_world
    Yeff_req = Eext.T * (e_inv * t_world_target)     # car Eext^{-1}=Eext^T (R ∈ SO(3))
    yref = S_YY.inv() * (Yeff_req - S_YX * xref)
    return xref, yref

# ---------- scénario complet ----------
def run_scenario(e, Rext, Rint, Eext, Eint,
                 SXX,SXY,SYX,SYY,
                 Xref_world, yref, n_digits=8):
    e_inv = e.inv()
    # opérateurs
    S = build_S(SXX,SXY,SYX,SYY)
    M = build_M(Rext,Rint,Eext,Eint)

    # sonde locale
    xref = e_inv * Xref_world
    f = sp.Matrix.vstack(xref, yref)

    # état effectif initial
    V0 = S * f
    Xeff, Yeff = V0[:3,:], V0[3:,:]

    # action
    V1 = M * V0
    X1_loc, Y1_loc = V1[:3,:], V1[3:,:]

    # lecture monde
    R_world = e * Rext * e_inv
    Xeff_world = e * Xeff
    t_world = e * (Eext * Yeff)
    X1_world = e * X1_loc

    # décomposition monde
    X1_world_pred = R_world * Xeff_world + t_world
    ok = bool(X1_world_pred.evalf(n_digits).equals(X1_world.evalf(n_digits)))

    # affichages
    print("\n=== Vérifs SO(3) ===")
    for name, R in [("R_ext",Rext),("R_int",Rint),("E_ext",Eext),("E_int",Eint)]:
        print(name, "∈ SO(3) ?", is_SO3(R))

    print("\n=== Vielbein e et e^{-1} ==="); display(e); display(e_inv)
    print("\n=== Blocs (tous dans SO(3)) ===")
    print("R_ext:"); display(Rext); print("R_int:"); display(Rint)
    print("E_ext:"); display(Eext); print("E_int:"); display(Eint)
    print("M = [[R_ext,E_ext],[E_int,R_int]] :"); display(M)

    print("\n=== Etat-opérateur S et sonde f ===")
    print("S:"); display(S); print("f (local) = [x_ref; y_ref] :"); display(f)

    print("\n=== Etat effectif initial V0 = S f ===")
    print("X_eff (loc):"); display(Xeff); print("Y_eff (loc):"); display(Yeff)
    print("X_eff (world):"); display(Xeff_world)

    print("\n=== Application V1 = M V0 ===")
    print("X1_loc:"); display(X1_loc); print("Y1_loc:"); display(Y1_loc)
    print("X1_world = e*X1_loc:"); display(X1_world)

    print("\n=== Décomposition monde ===")
    print("R_world = e R_ext e^{-1}:"); display(R_world)
    print("t_world = e (E_ext Y_eff):"); display(t_world)
    print("R_world*X_eff_world + t_world:"); display(X1_world_pred)
    print("Correspond à X1_world ? ->", ok)

    return dict(ok=ok, X1_world=X1_world, t_world=t_world, R_world=R_world)

# =========================
# Paramètres non triviaux
# =========================

# monde non-euclidien
e = sp.diag(2, 1, sp.Rational(1,2))

# 4 rotations indépendantes (toutes dans SO(3))
R_ext = euler_R(sp.pi/7,  sp.pi/6,  sp.pi/5,  "ZYX")
R_int = euler_R(sp.pi/10, sp.pi/9, -sp.pi/8,  "ZYX")
E_ext = euler_R(sp.pi/13, -sp.pi/12, sp.pi/11,"ZYX")   # "translation" externe = rotation
E_int = euler_R(-sp.pi/16, sp.pi/15, sp.pi/14,"ZYX")   # "translation" interne = rotation

# Etat-opérateur S (blocs : SYY ∈ SO(3) pour inversibilité propre)
SXX = sp.eye(3)
SXY = sp.Matrix([[0.2, 0.1, -0.3],
                 [0.0, -0.1, 0.2],
                 [0.1, 0.3, 0.0]])
SYX = sp.Matrix([[0.1, 0.0, 0.0],
                 [0.0, 0.1, 0.0],
                 [-0.1, 0.05, 0.05]])
SYY = R_int   # rotation interne (SO(3))

# Sonde : on VEUT une translation monde précise
Xref_world = sp.Matrix([3, -1, 2])
t_world_target = sp.Matrix([1, 2, 3])

# Calcule y_ref pour obtenir t_world_target avec ces R,E,S,e
xref, yref = yref_for_target_translation(e, R_ext, E_ext, SYY, SYX, Xref_world, t_world_target)

# Lance le scénario
res = run_scenario(e, R_ext, R_int, E_ext, E_int,
                   SXX,SXY,SYX,SYY,
                   Xref_world, yref, n_digits=8)



=== Vérifs SO(3) ===
R_ext ∈ SO(3) ? True
R_int ∈ SO(3) ? True
E_ext ∈ SO(3) ? True
E_int ∈ SO(3) ? True

=== Vielbein e et e^{-1} ===


⎡2  0   0 ⎤
⎢         ⎥
⎢0  1   0 ⎥
⎢         ⎥
⎣0  0  1/2⎦

⎡1/2  0  0⎤
⎢         ⎥
⎢ 0   1  0⎥
⎢         ⎥
⎣ 0   0  2⎦


=== Blocs (tous dans SO(3)) ===
R_ext:


⎡     ⎛1   √5⎞                                                                 ↪
⎢  √3⋅⎜─ + ──⎟          ________                                ________       ↪
⎢     ⎝4   4 ⎠         ╱ 5   √5     ⎛π⎞   ⎛1   √5⎞    ⎛π⎞      ╱ 5   √5     ⎛π ↪
⎢  ───────────    -   ╱  ─ - ── ⋅cos⎜─⎟ + ⎜─ + ──⎟⋅sin⎜─⎟     ╱  ─ - ── ⋅sin⎜─ ↪
⎢       2           ╲╱   8   8      ⎝7⎠   ⎝8   8 ⎠    ⎝7⎠   ╲╱   8   8      ⎝7 ↪
⎢                                                                              ↪
⎢       ________       ________                                                ↪
⎢      ╱ 5   √5       ╱ 5   √5     ⎛π⎞                                         ↪
⎢√3⋅  ╱  ─ - ──      ╱  ─ - ── ⋅sin⎜─⎟                                         ↪
⎢   ╲╱   8   8     ╲╱   8   8      ⎝7⎠   ⎛1   √5⎞    ⎛π⎞     ⎛1   √5⎞    ⎛π⎞   ↪
⎢───────────────   ─────────────────── + ⎜─ + ──⎟⋅cos⎜─⎟   - ⎜─ + ──⎟⋅sin⎜─⎟ + ↪
⎢       2                   2            ⎝4   4 ⎠    ⎝7⎠     ⎝4   4 ⎠    ⎝7⎠   ↪
⎢                           

...

t_world = e (E_ext Y_eff):


⎡                                                    ⎛                         ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎜⎛             ________   ↪
⎢  ⎛     ⎛π ⎞    ⎛π ⎞   ⎛  √6   √2⎞    ⎛π ⎞    ⎛π ⎞⎞ ⎜⎜⎛1   √5⎞    ╱ √2   1    ↪
⎢2⋅⎜- sin⎜──⎟⋅cos⎜──⎟ + ⎜- ── + ──⎟⋅sin⎜──⎟⋅cos⎜──⎟⎟⋅⎜⎜⎜─ - ──⎟⋅  ╱  ── + ─  - ↪
⎢  ⎝     ⎝11⎠    ⎝13⎠   ⎝  4    4 ⎠    ⎝13⎠    ⎝11⎠⎠ ⎜⎝⎝4   4 ⎠ ╲╱   4    2    ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎝                         ↪
⎢                                                                              ↪
⎢                                                     ⎛  ⎛  ⎛                  ↪
⎢                                                     ⎜  ⎜  ⎜                  ↪
⎢                           

R_world*X_eff_world + t_world:


⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                    ⎛                         ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎜                         ↪
⎢                                                    ⎜⎛             ________   ↪
⎢  ⎛     ⎛π ⎞    ⎛π ⎞   ⎛  √6   √2⎞    ⎛π ⎞    ⎛π ⎞⎞ ⎜⎜⎛1   √5⎞    ╱ √2   1    ↪
⎢2⋅⎜- sin⎜──⎟⋅cos⎜──⎟ + ⎜- ── + ──⎟⋅sin⎜──⎟⋅cos⎜──⎟⎟⋅⎜⎜⎜─ - ──⎟⋅  ╱  ── + ─  - ↪
⎢  ⎝     ⎝11⎠    ⎝13⎠   ⎝  4    4 ⎠    ⎝13⎠    ⎝11⎠⎠ ⎜⎝⎝4   4 ⎠ ╲╱   4    2    ↪
⎢                                                    ⎜                         ↪
⎢                           

Correspond à X1_world ? -> True


In [5]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# =========================
# 1) UTILITAIRES
# =========================

def Rx(a):
    ca, sa = sp.cos(a), sp.sin(a)
    return sp.Matrix([[1,0,0],[0,ca,-sa],[0,sa,ca]])

def Ry(a):
    ca, sa = sp.cos(a), sp.sin(a)
    return sp.Matrix([[ca,0,sa],[0,1,0],[-sa,0,ca]])

def Rz(a):
    ca, sa = sp.cos(a), sp.sin(a)
    return sp.Matrix([[ca,-sa,0],[sa,ca,0],[0,0,1]])

def euler_R(roll, pitch, yaw, order="ZYX"):
    if order.upper()=="ZYX":
        return Rz(yaw)*Ry(pitch)*Rx(roll)
    elif order.upper()=="XYZ":
        return Rx(roll)*Ry(pitch)*Rz(yaw)
    else:
        raise ValueError("Ordre non supporté (utilise 'ZYX' ou 'XYZ').")

def build_block_from_world_cols(e_inv, cols_world):
    """Construit un bloc 3x3 dont les colonnes locales sont e^{-1}*col_world."""
    cols_local = [e_inv * c for c in cols_world]
    return sp.Matrix.hstack(*cols_local)

def apply_operator(e, R_ext, R_int, E_ext, E_int, X0_world, Y0, n_digits=6):
    """Applique M sur V0, affiche toutes les étapes, et renvoie un dict de résultats."""
    e_inv = e.inv()
    Z = sp.zeros(3)
    M = sp.Matrix.vstack(
            sp.Matrix.hstack(R_ext, E_ext),
            sp.Matrix.hstack(E_int, R_int)
        )
    # Projections
    X0_loc = e_inv * X0_world
    V0 = sp.Matrix.vstack(X0_loc, Y0)
    V1 = M * V0
    X1_loc = V1[:3, :]
    Y1_loc = V1[3:, :]
    # Lecture monde
    X1_world = e * X1_loc
    R_world = e * R_ext * e_inv
    t_world = e * (E_ext * Y0)

    # Affichages
    print("\n=== Vielbein e (local -> monde) et inverse ===")
    display(e); display(e_inv)

    print("\n=== Blocs de ROTATION ===")
    print("R_ext :"); display(R_ext)
    print("R_int :"); display(R_int)

    print("\n=== Blocs de TRANSLATION (carrés 3x3) ===")
    print("E_ext :"); display(E_ext)
    print("E_int :"); display(E_int)

    print("\n=== Super-matrice M (6x6) ===")
    display(M)

    print("\n=== Etat initial ===")
    print("X0_world :"); display(X0_world)
    print("X0_loc = e^{-1} X0_world :"); display(X0_loc)
    print("Y0 (slots) :"); display(Y0)
    print("V0 = [X0_loc ; Y0] :"); display(V0)

    print("\n=== Application locale : V1 = M * V0 ===")
    print("V1 :"); display(V1)
    print("X1_loc (partie haute) :"); display(X1_loc)
    print("Y1_loc (partie basse) :"); display(Y1_loc)

    print("\n=== Lecture MONDE et interprétation ===")
    print("R_world = e * R_ext * e^{-1} :"); display(R_world)
    print("t_world = e * (E_ext * Y0) :"); display(t_world)
    print("X1_world = e * X1_loc :"); display(X1_world)
    print("Vérif : X1_world ?= R_world * X0_world + t_world")
    display((R_world*X0_world + t_world).evalf(n_digits))
    display(X1_world.evalf(n_digits))
    print("Egalité numérique (à ~1e-{} près) : {}".format(n_digits, 
          bool((R_world*X0_world + t_world).evalf(n_digits).equals(X1_world.evalf(n_digits)))))

    return {
        "M": M, "X0_world": X0_world, "X0_loc": X0_loc, "Y0": Y0,
        "V0": V0, "V1": V1, "X1_loc": X1_loc, "Y1_loc": Y1_loc,
        "X1_world": X1_world, "R_world": R_world, "t_world": t_world
    }

# =========================
# 2) PARAMETRES D'UN EXEMPLE NON TRIVIAL
# =========================

# Vielbein (non euclidien global)
e = sp.diag(2, 1, sp.Rational(1,2))      # étire x, identique y, comprime z
e_inv = e.inv()

# Rotations locales (différentes)
R_ext = euler_R(roll=sp.pi/7, pitch=sp.pi/6, yaw=sp.pi/5, order="ZYX")
R_int = euler_R(roll=sp.pi/10, pitch=sp.pi/9, yaw=-sp.pi/8, order="ZYX")

# Colonnes MONDE ciblées pour la translation externe (distinctes)
tW1 = sp.Matrix([1, 2, 0])
tW2 = sp.Matrix([0, -1, 1])
tW3 = sp.Matrix([2, 0, 3])
E_ext = build_block_from_world_cols(e_inv, [tW1, tW2, tW3])

# Colonnes MONDE ciblées pour la translation interne (couplage retour)
sW1 = sp.Matrix([sp.Rational(1,2), -sp.Rational(1,2), 0])
sW2 = sp.Matrix([0, 1, -1])
sW3 = sp.Matrix([1, 0, -sp.Rational(1,2)])
E_int = build_block_from_world_cols(e_inv, [sW1, sW2, sW3])

# Slots activés (non triviaux)
Y0 = sp.Matrix([sp.Rational(3,10), 1, -sp.Rational(1,2)])

# Position initiale monde (non triviale)
X0_world = sp.Matrix([3, -1, 2])

# =========================
# 3) LANCER L'EXEMPLE
# =========================
res = apply_operator(e, R_ext, R_int, E_ext, E_int, X0_world, Y0, n_digits=8)

# =========================
# 4) COMMENT FAIRE D'AUTRES EXEMPLES ?
# -------------------------
# - Change e (et e_inv), R_ext, R_int (angles), les colonnes monde désirées (tW*, sW*),
#   puis regénère E_ext, E_int via build_block_from_world_cols.
# - Change Y0 (slots), X0_world (position initiale).
# - Relance apply_operator(...)



=== Vielbein e (local -> monde) et inverse ===


⎡2  0   0 ⎤
⎢         ⎥
⎢0  1   0 ⎥
⎢         ⎥
⎣0  0  1/2⎦

⎡1/2  0  0⎤
⎢         ⎥
⎢ 0   1  0⎥
⎢         ⎥
⎣ 0   0  2⎦


=== Blocs de ROTATION ===
R_ext :


⎡     ⎛1   √5⎞                                                                 ↪
⎢  √3⋅⎜─ + ──⎟          ________                                ________       ↪
⎢     ⎝4   4 ⎠         ╱ 5   √5     ⎛π⎞   ⎛1   √5⎞    ⎛π⎞      ╱ 5   √5     ⎛π ↪
⎢  ───────────    -   ╱  ─ - ── ⋅cos⎜─⎟ + ⎜─ + ──⎟⋅sin⎜─⎟     ╱  ─ - ── ⋅sin⎜─ ↪
⎢       2           ╲╱   8   8      ⎝7⎠   ⎝8   8 ⎠    ⎝7⎠   ╲╱   8   8      ⎝7 ↪
⎢                                                                              ↪
⎢       ________       ________                                                ↪
⎢      ╱ 5   √5       ╱ 5   √5     ⎛π⎞                                         ↪
⎢√3⋅  ╱  ─ - ──      ╱  ─ - ── ⋅sin⎜─⎟                                         ↪
⎢   ╲╱   8   8     ╲╱   8   8      ⎝7⎠   ⎛1   √5⎞    ⎛π⎞     ⎛1   √5⎞    ⎛π⎞   ↪
⎢───────────────   ─────────────────── + ⎜─ + ──⎟⋅cos⎜─⎟   - ⎜─ + ──⎟⋅sin⎜─⎟ + ↪
⎢       2                   2            ⎝4   4 ⎠    ⎝7⎠     ⎝4   4 ⎠    ⎝7⎠   ↪
⎢                           

...

⎡                                                                              ↪
⎢                                 ________                ________          3⋅ ↪
⎢  7      ⎛1   √5⎞    ⎛π⎞        ╱ 5   √5     ⎛π⎞        ╱ 5   √5     ⎛π⎞      ↪
⎢- ── - 2⋅⎜─ + ──⎟⋅sin⎜─⎟ + 2⋅  ╱  ─ - ── ⋅cos⎜─⎟ + 8⋅  ╱  ─ - ── ⋅sin⎜─⎟ + ── ↪
⎢  10     ⎝8   8 ⎠    ⎝7⎠     ╲╱   8   8      ⎝7⎠     ╲╱   8   8      ⎝7⎠      ↪
⎢                                                                              ↪
⎢                                                ________                   __ ↪
⎢                                               ╱ 5   √5     ⎛π⎞           ╱ 5 ↪
⎢                                              ╱  ─ - ── ⋅sin⎜─⎟   3⋅√3⋅  ╱  ─ ↪
⎢    ⎛1   √5⎞    ⎛π⎞   ⎛1   √5⎞    ⎛π⎞   2   ╲╱   8   8      ⎝7⎠        ╲╱   8 ↪
⎢- 4⋅⎜─ + ──⎟⋅sin⎜─⎟ - ⎜─ + ──⎟⋅cos⎜─⎟ - ─ - ─────────────────── + ─────────── ↪
⎢    ⎝4   4 ⎠    ⎝7⎠   ⎝4   4 ⎠    ⎝7⎠   5            2                    4   ↪
⎢                           

Vérif : X1_world ?= R_world * X0_world + t_world


⎡ 7.0658611 ⎤
⎢           ⎥
⎢-0.83778376⎥
⎢           ⎥
⎣0.49764669 ⎦

⎡ 7.0658611 ⎤
⎢           ⎥
⎢-0.83778376⎥
⎢           ⎥
⎣0.49764669 ⎦

Egalité numérique (à ~1e-8 près) : True


In [4]:
import sympy as sp
sp.init_printing()

# Vielbein e : local -> monde (non euclidien global)
e = sp.diag(2, 1, sp.Rational(1,2)); e_inv = e.inv()

# Rotations locales (identité)
R_ext = sp.eye(3); R_int = sp.eye(3)

# Translation monde désirée et conversion en local
t_world = sp.Matrix([1,2,3])
t_loc   = e_inv * t_world

# Slot idéal activé Y0
Y0 = sp.Matrix([0,0,1])

# Bloc E_ext : colonne 3 = t_loc  (rotation à centre infini -> translation)
E_ext = sp.zeros(3); E_ext[:,2] = t_loc
E_int = sp.zeros(3)

# Super-opérateur 6×6 : [[R_ext, E_ext],[E_int, R_int]]
Z = sp.zeros(3)
M = sp.Matrix.vstack(sp.Matrix.hstack(R_ext, E_ext),
                     sp.Matrix.hstack(E_int, R_int))

# État initial en monde puis local
X0_world = sp.Matrix([10,20,30]); X0_local = e_inv*X0_world
V0 = sp.Matrix.vstack(X0_local, Y0)

# Application purement matricielle
V1 = M*V0
X1_local = V1[:3,:]; X1_world = e*X1_local

# Vérification : X1_world == X0_world + t_world
print("OK ?", bool(X1_world == X0_world + t_world))


OK ? True


In [3]:
import sympy as sp
sp.init_printing()

# Vielbein (local->monde) : espace global non euclidien
e = sp.diag(2, 1, sp.Rational(1,2)); e_inv = e.inv()

# Rotations locales (identité ici, mets Euler si tu veux)
R_ext = sp.eye(3); R_int = sp.eye(3)

# Translation monde désirée et conversion en local
t_world = sp.Matrix([1,2,3])
t_loc   = e_inv * t_world

# Slot idéal activé (direction à l'infini choisie)
Y0 = sp.Matrix([0,0,1])

# E_ext : colonne 3 = t_loc  -> "rotation de centre à l'infini" = translation
E_ext = sp.zeros(3); E_ext[:,2] = t_loc
E_int = sp.zeros(3)  # garde 0 pour la démo (tu peux l'activer ensuite)

# Super-opérateur 6x6
Z = sp.zeros(3)
M = sp.Matrix.vstack(
      sp.Matrix.hstack(R_ext, E_ext),
      sp.Matrix.hstack(E_int, R_int)
    )

# Etat initial en monde puis local
X0_world = sp.Matrix([10,20,30]); X0_loc = e_inv*X0_world
V0 = sp.Matrix.vstack(X0_loc, Y0)
V1 = M*V0
X1_world = e * V1[:3,:]

# Vérification : translation habituelle récupérée
assert X1_world == X0_world + t_world


In [1]:
import sympy as sp
from IPython.display import display

sp.init_printing()

# ------------------------------------------------------------
# 1) Données du "monde" et du "local"
# ------------------------------------------------------------
# Repère local -> monde (vielbein/coframe) : e (3x3 inversible)
# e = I  => espace local = monde (euclidien local trivial)
# Choisis un e non trivial pour "non-euclidien global" (anisotropie locale)
e = sp.diag(2, 1, sp.Rational(1,2))   # exemple : étire x, identique y, comprime z
e_inv = e.inv()

print("Vielbein e (local -> monde) :")
display(e)
print("e^{-1} :")
display(e_inv)

# ------------------------------------------------------------
# 2) Rotations locale externes / internes (ici identité, modifie si besoin)
# ------------------------------------------------------------
R_ext = sp.eye(3)
R_int = sp.eye(3)

print("R_ext (rotation externe locale) :")
display(R_ext)
print("R_int (rotation interne locale) :")
display(R_int)

# ------------------------------------------------------------
# 3) Translation externe/interne sous forme de BLOC 3x3 (pas de vector-colonne)
#    On encode une translation locale t_loc en mettant t_loc comme une colonne de E_ext.
#    Slot d'activation Y0 = e3 = (0,0,1) => E_ext * Y0 = (colonne 3 de E_ext) = t_loc
# ------------------------------------------------------------
# Translation voulue en MONDE : t_world = (1,2,3)
t_world = sp.Matrix([1,2,3])
# On convertit en LOCAL : t_loc = e^{-1} * t_world
t_loc = e_inv * t_world

print("Translation MONDE désirée t_world :")
display(t_world)
print("Translation LOCALE correspondante t_loc = e^{-1} * t_world :")
display(t_loc)

# Choix du slot d'activation dans Y0 (ici e3)
Y0 = sp.Matrix([0,0,1])

# Construire E_ext en plaçant t_loc comme 3e colonne
E_ext = sp.zeros(3)
E_ext[:,2] = t_loc  # colonne 3 = t_loc
print("Bloc translation externe E_ext (3e colonne = t_loc) :")
display(E_ext)

# Optionnel: une translation "interne" E_int (retour/couplage vers Y)
# Ici on met E_int = 0 pour reproduire la translation simple; tu peux l'activer ensuite.
E_int = sp.zeros(3)
print("Bloc translation interne E_int :")
display(E_int)

# ------------------------------------------------------------
# 4) Super-opérateur 6x6 local : M = [[R_ext, E_ext],[E_int, R_int]]
# ------------------------------------------------------------
Z = sp.zeros(3)
M = sp.Matrix.vstack(
        sp.Matrix.hstack(R_ext, E_ext),
        sp.Matrix.hstack(E_int, R_int)
)
print("Super-matrice locale M (6x6) :")
display(M)

# ------------------------------------------------------------
# 5) Etat initial : X0 (position locale) et Y0 (slot)
#    - On choisit X0_world, on le convertit en local via e^{-1}
# ------------------------------------------------------------
X0_world = sp.Matrix([10,20,30])
X0_local = e_inv * X0_world

print("Position monde initiale X0_world :")
display(X0_world)
print("Position locale initiale X0_local = e^{-1} * X0_world :")
display(X0_local)

# Etat total local V0 = [X0_local ; Y0]
V0 = sp.Matrix.vstack(X0_local, Y0)
print("Etat local initial V0 = [X0_local ; Y0] :")
display(V0)

# ------------------------------------------------------------
# 6) Application purement matricielle locale
# ------------------------------------------------------------
V1 = M * V0
X1_local = V1[:3, :]
Y1_local = V1[3:, :]

print("Etat local final V1 = M * V0 :")
display(V1)
print("Partie haute locale (position) X1_local :")
display(X1_local)
print("Partie basse locale (slot)     Y1_local :")
display(Y1_local)

# ------------------------------------------------------------
# 7) Projection en MONDE et vérifications
# ------------------------------------------------------------
X1_world = e * X1_local
X0_plus_t_world = X0_world + t_world

print("Position monde finale X1_world = e * X1_local :")
display(X1_world)
print("Comparaison avec translation 'habituelle' X0_world + t_world :")
display(X0_plus_t_world)

print("Vérification (R=I, E_int=0) :  X1_world == X0_world + t_world  ?  →",
      bool(X1_world == X0_plus_t_world))


Vielbein e (local -> monde) :


⎡2  0   0 ⎤
⎢         ⎥
⎢0  1   0 ⎥
⎢         ⎥
⎣0  0  1/2⎦

e^{-1} :


⎡1/2  0  0⎤
⎢         ⎥
⎢ 0   1  0⎥
⎢         ⎥
⎣ 0   0  2⎦

R_ext (rotation externe locale) :


...

Position monde finale X1_world = e * X1_local :


⎡11⎤
⎢  ⎥
⎢22⎥
⎢  ⎥
⎣33⎦

Comparaison avec translation 'habituelle' X0_world + t_world :


⎡11⎤
⎢  ⎥
⎢22⎥
⎢  ⎥
⎣33⎦

Vérification (R=I, E_int=0) :  X1_world == X0_world + t_world  ?  → True


# Matrice antisymétrique 6×6 : base parfaitement symétrique rotation/translation

Nous considérons ici une base :
- f₁ : rotation autour de x
- f₂ : translation selon x
- f₃ : rotation autour de y
- f₄ : translation selon y
- f₅ : rotation autour de z
- f₆ : translation selon z

**Chaque coefficient** de la matrice encode une “interaction” entre deux générateurs :  
soit une rotation pure, soit une translation pure, soit un couplage hélicoïdal.
La matrice antisymétrique permet de modéliser de façon parfaitement équitable et symétrique tous les mouvements rigides de l’espace (SE(3)),  
et c’est la structure naturelle dans laquelle la translation et la rotation sont “duales” et échangeables.


In [1]:
import sympy as sp
from sympy import symbols, Matrix

# Définition des symboles (valeurs arbitraires pour la démo)
θx, θy, θz = symbols('theta_x theta_y theta_z')
tx, ty, tz = symbols('t_x t_y t_z')
α14, α25, α36 = symbols('alpha_14 alpha_25 alpha_36')  # Exemples de couplages rotation/translation

# Création de la matrice antisymétrique 6x6
M = sp.zeros(6, 6)
# Rotations pures (blocs usuels)
M[0,2] = θx; M[2,0] = -θx      # f1^f3, rot x
M[2,4] = θy; M[4,2] = -θy      # f3^f5, rot y
M[4,0] = θz; M[0,4] = -θz      # f5^f1, rot z
# Translations pures (blocs usuels)
M[1,3] = tx; M[3,1] = -tx      # f2^f4, trans x
M[3,5] = ty; M[5,3] = -ty      # f4^f6, trans y
M[5,1] = tz; M[1,5] = -tz      # f6^f2, trans z
# Exemples de couplage rotation/translation (hélicoïdal hors axes)
M[0,3] = α14; M[3,0] = -α14    # f1^f4, rot x + trans y
M[1,4] = α25; M[4,1] = -α25    # f2^f5, trans x + rot z
M[2,5] = α36; M[5,2] = -α36    # f3^f6, rot y + trans z

# Affichage latex
display(sp.pretty(M))
display(sp.Matrix(M))

# Affichage LaTeX bien mis en forme
sp.init_printing()
display(M)


'⎡ 0     0     θₓ   α₁₄   -θ_z   0  ⎤\n⎢                                  ⎥\n⎢ 0     0     0     tₓ   α₂₅   -t_z⎥\n⎢                                  ⎥\n⎢-θₓ    0     0     0    θ_y   α₃₆ ⎥\n⎢                                  ⎥\n⎢-α₁₄  -tₓ    0     0     0    t_y ⎥\n⎢                                  ⎥\n⎢θ_z   -α₂₅  -θ_y   0     0     0  ⎥\n⎢                                  ⎥\n⎣ 0    t_z   -α₃₆  -t_y   0     0  ⎦'

Matrix([
[        0,         0,   theta_x, alpha_14, -theta_z,        0],
[        0,         0,         0,      t_x, alpha_25,     -t_z],
[ -theta_x,         0,         0,        0,  theta_y, alpha_36],
[-alpha_14,      -t_x,         0,        0,        0,      t_y],
[  theta_z, -alpha_25,  -theta_y,        0,        0,        0],
[        0,       t_z, -alpha_36,     -t_y,        0,        0]])

⎡ 0     0     θₓ   α₁₄   -θ_z   0  ⎤
⎢                                  ⎥
⎢ 0     0     0     tₓ   α₂₅   -t_z⎥
⎢                                  ⎥
⎢-θₓ    0     0     0    θ_y   α₃₆ ⎥
⎢                                  ⎥
⎢-α₁₄  -tₓ    0     0     0    t_y ⎥
⎢                                  ⎥
⎢θ_z   -α₂₅  -θ_y   0     0     0  ⎥
⎢                                  ⎥
⎣ 0    t_z   -α₃₆  -t_y   0     0  ⎦

In [8]:
# Dictionnaire d'interprétation de la base
base = {
    0: "f₁ : rotation autour de x",
    1: "f₂ : translation selon x",
    2: "f₃ : rotation autour de y",
    3: "f₄ : translation selon y",
    4: "f₅ : rotation autour de z",
    5: "f₆ : translation selon z"
}

# Explications détaillées
explications = []
for i in range(6):
    for j in range(i+1, 6):
        if M[i,j] != 0:
            expl = f"{sp.latex(M[i,j])} ({base[i]} ∧ {base[j]})"
            explications.append(expl)
        elif M[j,i] != 0:
            expl = f"-{sp.latex(M[j,i])} ({base[j]} ∧ {base[i]})"
            explications.append(expl)

print("Interprétation géométrique des coefficients non nuls :\n")
for e in explications:
    print("•", e)


Interprétation géométrique des coefficients non nuls :

• \theta_{x} (f₁ : rotation autour de x ∧ f₃ : rotation autour de y)
• \alpha_{14} (f₁ : rotation autour de x ∧ f₄ : translation selon y)
• - \theta_{z} (f₁ : rotation autour de x ∧ f₅ : rotation autour de z)
• t_{x} (f₂ : translation selon x ∧ f₄ : translation selon y)
• \alpha_{25} (f₂ : translation selon x ∧ f₅ : rotation autour de z)
• - t_{z} (f₂ : translation selon x ∧ f₆ : translation selon z)
• \theta_{y} (f₃ : rotation autour de y ∧ f₅ : rotation autour de z)
• \alpha_{36} (f₃ : rotation autour de y ∧ f₆ : translation selon z)
• t_{y} (f₄ : translation selon y ∧ f₆ : translation selon z)


# Résumé

- **Les blocs (1,3), (3,5), (5,1) :** rotations autour des axes x, y, z.
- **Les blocs (2,4), (4,6), (6,2) :** translations selon x, y, z.
- **Les autres coefficients** (ex : (1,4), (2,5), (3,6)) modélisent des couplages rotation/translation :  
  ce sont des mouvements hélicoïdaux hors des axes principaux (par exemple une vis dont l’axe est oblique).
- **La structure antisymétrique** garantit que la matrice code un mouvement rigide (aucune distorsion de l’espace).
- Cette structure permet d’interpréter la **dualité rotation/translation** :  
  c’est la base qui détermine l’interprétation de chaque coefficient.

Vous pouvez changer les symboles, en ajouter, mettre des valeurs numériques,
et afficher la matrice pour explorer d’autres mouvements ou couplages !


Préambule miniature

In [4]:
import numpy as np
from scipy.linalg import expm

# Valeurs numériques
theta_x, theta_y, theta_z = 0.3, 0.4, 0.5
t_x, t_y, t_z = 1.1, 1.2, 1.3

M = np.zeros((6, 6))
M[0, 2] = theta_x;  M[2, 0] = -theta_x
M[2, 4] = theta_y;  M[4, 2] = -theta_y
M[4, 0] = theta_z;  M[0, 4] = -theta_z
M[1, 3] = t_x;      M[3, 1] = -t_x
M[3, 5] = t_y;      M[5, 3] = -t_y
M[5, 1] = t_z;      M[1, 5] = -t_z

T = expm(M)

# Affichage arrondi
np.set_printoptions(precision=4, suppress=True)
print("Matrice exponentielle T = exp(M):\n", T)


Matrice exponentielle T = exp(M):
 [[ 0.837   0.      0.3715  0.     -0.4018  0.    ]
 [ 0.      0.0042  0.      0.9959  0.     -0.0906]
 [-0.1797  0.      0.8801  0.      0.4394  0.    ]
 [ 0.      0.0755  0.      0.09    0.      0.9931]
 [ 0.5169  0.     -0.2956  0.      0.8034  0.    ]
 [ 0.      0.9971  0.     -0.011   0.     -0.0748]]


# Mouvements rigides en 3D : une matrice antisymétrique 6×6 “pleine”

Dans cette présentation, nous modélisons une transformation rigide (rotation + translation, voire combinaison hélicoïdale) de l’espace 3D
sous forme d’une matrice **antisymétrique 6×6**, dans une base où rotation et translation sont traitées de façon parfaitement équivalente.

## Base choisie :

- f₁ : rotation autour de x
- f₂ : translation selon x
- f₃ : rotation autour de y
- f₄ : translation selon y
- f₅ : rotation autour de z
- f₆ : translation selon z

**Chaque coefficient** de la matrice encode une “interaction” entre deux générateurs.
En pratique, cette base fait apparaître la dualité parfaite entre rotation et translation,
et permet de modéliser tous les mouvements rigides (ou hélicoïdaux) de l’espace.

Vous pouvez choisir n'importe quelle combinaison de rotations et translations !


In [1]:
import numpy as np
from scipy.linalg import expm

def build_m6x6(theta_x=0, theta_y=0, theta_z=0, t_x=0, t_y=0, t_z=0):
    """
    Construit la matrice antisymétrique 6x6 pour :
    - theta_x, theta_y, theta_z : rotations autour de x, y, z
    - t_x, t_y, t_z : translations selon x, y, z
    """
    M = np.zeros((6, 6))
    # Rotations
    M[0, 2] = theta_x; M[2, 0] = -theta_x   # f1^f3 = rotation x
    M[2, 4] = theta_y; M[4, 2] = -theta_y   # f3^f5 = rotation y
    M[4, 0] = theta_z; M[0, 4] = -theta_z   # f5^f1 = rotation z
    # Translations
    M[1, 3] = t_x;     M[3, 1] = -t_x       # f2^f4 = translation x
    M[3, 5] = t_y;     M[5, 3] = -t_y       # f4^f6 = translation y
    M[5, 1] = t_z;     M[1, 5] = -t_z       # f6^f2 = translation z
    return M

def explain_M(M):
    """
    Retourne l'expression symbolique des coefficients non nuls de la matrice M,
    en rappelant leur interprétation géométrique.
    """
    interpretation = []
    base_names = [
        "f₁: rot x", "f₂: trans x", "f₃: rot y", 
        "f₄: trans y", "f₅: rot z", "f₆: trans z"
    ]
    for i in range(6):
        for j in range(i+1, 6):
            if not np.isclose(M[i, j], 0) or not np.isclose(M[j, i], 0):
                interpretation.append(
                    f"M[{i+1},{j+1}] = {M[i,j]:.3f}  : {base_names[i]} ∧ {base_names[j]}"
                )
    return "\n".join(interpretation)

def show_matrix(M, title="Matrice antisymétrique 6×6"):
    print(title)
    print(np.array2string(M, precision=3, suppress_small=True))

def show_exp(M):
    """
    Affiche l'exponentielle de la matrice 6x6 (transformation finie)
    """
    expM = expm(M)
    print("Exponentielle exp(M) (transformation finie) :")
    print(np.array2string(expM, precision=3, suppress_small=True))
    return expM


In [2]:
# Choisissez ici vos valeurs de rotations et translations !
theta_x = 0.3  # Rotation autour de x
theta_y = 0.4  # Rotation autour de y
theta_z = 0.5  # Rotation autour de z
t_x = 1.1      # Translation selon x
t_y = 1.2      # Translation selon y
t_z = 1.3      # Translation selon z

# Construction de la matrice
M = build_m6x6(theta_x, theta_y, theta_z, t_x, t_y, t_z)

# Affichage de la matrice
show_matrix(M, title="Votre matrice antisymétrique 6x6 :")

# Explication symbolique de chaque coefficient non nul
print("\nInterprétation géométrique des coefficients non nuls :")
print(explain_M(M))


Votre matrice antisymétrique 6x6 :
[[ 0.   0.   0.3  0.  -0.5  0. ]
 [ 0.   0.   0.   1.1  0.  -1.3]
 [-0.3  0.   0.   0.   0.4  0. ]
 [ 0.  -1.1  0.   0.   0.   1.2]
 [ 0.5  0.  -0.4  0.   0.   0. ]
 [ 0.   1.3  0.  -1.2  0.   0. ]]

Interprétation géométrique des coefficients non nuls :
M[1,3] = 0.300  : f₁: rot x ∧ f₃: rot y
M[1,5] = -0.500  : f₁: rot x ∧ f₅: rot z
M[2,4] = 1.100  : f₂: trans x ∧ f₄: trans y
M[2,6] = -1.300  : f₂: trans x ∧ f₆: trans z
M[3,5] = 0.400  : f₃: rot y ∧ f₅: rot z
M[4,6] = 1.200  : f₄: trans y ∧ f₆: trans z


In [3]:
# Calcul et affichage de l'exponentielle
T = show_exp(M)

# Note : L'exponentielle de cette matrice représente
# la transformation composée (rotation/translation/hélicoïdale)
# correspondant à vos choix de coefficients.
# Les blocs diagonaux sont liés aux rotations "combinées".
# Les blocs hors diagonale à l'influence croisée entre rotations et translations.


Exponentielle exp(M) (transformation finie) :
[[ 0.837  0.     0.372  0.    -0.402  0.   ]
 [ 0.     0.004  0.     0.996  0.    -0.091]
 [-0.18   0.     0.88   0.     0.439  0.   ]
 [ 0.     0.076  0.     0.09   0.     0.993]
 [ 0.517  0.    -0.296  0.     0.803  0.   ]
 [ 0.     0.997  0.    -0.011  0.    -0.075]]


# Notes d'interprétation

- **Chaque coefficient** de la matrice 6x6 correspond à un générateur du mouvement rigide :
    - Les blocs (1,3), (3,5), (5,1) : rotations autour de x, y, z
    - Les blocs (2,4), (4,6), (6,2) : translations selon x, y, z
- **La structure antisymétrique** assure que la transformation est "rigide" (aucune distorsion de l'espace).
- **L’exponentielle de la matrice** donne la transformation globale finie
  (composition non triviale de rotations et translations, donc potentiellement hélicoïdale).

Vous pouvez modifier les valeurs pour tester d’autres mouvements !
Ce formalisme place **rotations et translations sur un même plan mathématique**,
et permet de composer, mélanger, et permuter ces mouvements sans distinction structurelle.

**Ce formalisme est très puissant pour l’étude de la cinématique, de la robotique, ou pour les fondements géométriques.**


# Aller plus loin

- On peut inclure d'autres coefficients pour explorer le couplage rotation-translation hors axes principaux.
- Les coefficients peuvent être pris dans C, R ou même dans des extensions (complexes, duaux…) pour explorer la géométrie conforme ou projective.
- L’exponentielle peut être utilisée pour interpoler entre mouvements ou pour modéliser des mouvements continus (vis, hélices…).
- Ce formalisme permet aussi d’étudier la structure de Lie de SE(3), et d’explorer la dualité rotation/translation (changement de base, symétrie).

N'hésitez pas à faire vos propres essais et à modifier les coefficients !
